In [2]:
import torch
import torchvision
# import
import os
import pickle
import numpy as np
import PIL.Image

import scipy.io as sio
from datetime import datetime
import sys


sys.path.append('../model')
from resnet3d import resnet, mean
                
sys.path.append('../cnn_preferred')
from utils import normalise_img, clip_extreme_pixel, save_video, normalise_vid, get_target_feature_shape, save_gif
from activation_maximization import generate_preferred


In [ ]:
net = resnet.resnet50(num_classes=400, shortcut_type='B', sample_size=112, sample_duration=16)
net = torch.nn.DataParallel(net, device_ids=None)
param_file = os.path.join('../model','resnet3d', 'resnet-50-kinetics.pth')
param = torch.load(param_file, map_location='cpu')
net.load_state_dict(param['state_dict'])
net = net.module
net.eval()


In [ ]:
img_mean = np.float32([104, 117, 123])
img_std = np.float32([1,1,1])

# image
img_mean=np.array([0.485, 0.456, 0.406],dtype=np.float),
img_std = np.array([0.229,0.224,0.225])

img_mean = mean.get_mean(1)
img_std = mean.get_std(1)
norm = 255

In [ ]:
img_std

In [5]:
#save_dir
save_dir = '../result'
save_folder = 'jupyter_demo_torch_resnet3D_TV_norm_all'#__file__.split('.')[0]
save_folder = save_folder + '_' + datetime.now().strftime('%Y%m%dT%H%M%S')
save_path = os.path.join(save_dir,save_folder)
os.makedirs(save_path, exist_ok=True)

In [6]:
def hook(module, input, output):
    outputs.append(output.clone())

In [7]:
# initial image for the optimization
fr, h, w = 16, 112,112

initial_input = np.random.randint(0, 256, (fr, h,w,3))

In [8]:
net

ResNet(
  (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (iden): MyIdentity()
      (downsample): Sequential(
     

In [22]:
target_layer = "net.layer1[2].conv3"
target_layer = "module.conv1"
target_layer = "fc"
#target_layer = "net.module.layer4[0].bn3"
target_layer = "layer4[2].conv3"

In [23]:
#targetlayer
outputs = []

target_layer_list = [target_layer]
feat_num = get_target_feature_shape(net, torch.Tensor(initial_input.transpose(3,0, 1, 2)[np.newaxis]), target_layer_list)



In [28]:
feat_num

(1, 2048, 1, 4, 4)

In [24]:
fr, h, w = 16, 112,112
initial_input = np.zeros((fr, h,w,3),dtype='float32')

# options
opts = {
    'img_mean': img_mean,
    'img_std': img_std,

    'iter_n': 200, # the total number of iterations for gradient descend

    'disp_every': 1, # display the information on the terminal for every n iterations

    'save_intermediate': True, # save the intermediate or not
    'save_intermediate_every': 10, # save the intermediate for every n iterations
    'save_intermediate_path': save_path, # the path to save the intermediate

    'lr_start': 1., # learning rate
    'lr_end': 1.,

    'momentum_start': 0.001, # gradient with momentum
    'momentum_end': 0.001,

    'decay_start': 0.001, # pixel decay for each iteration
    'decay_end': 0.001,

    'image_blur': True, # Use image smoothing or not
    'sigma_start': 2.5, # the size of the gaussian filter for image smoothing
    'sigma_end': 0.5,

    'image_jitter': True, # use image jittering during
    'jitter_size': 4,
    
    'use_p_norm_reg': False,
    'p': 2,

    'use_TV_norm_reg': False,
    'TVbeta1': 1, 
    'TVbeta2':1.2,

    'clip_small_norm': True,
    'clip_small_norm_every': 1,
    'n_pct_start': 5,
    'n_pct_end': 5,

    'clip_small_contribution': True,
    'clip_small_contribution_every': 1,
    'c_pct_start': 5,
    'c_pct_end':5,
    
    'input_size': (16, 112,112,3),
    #'input_size': (224,224,3),
    #'initial_input': None, # the initial image for the optimization (setting to None will use random noise as initial image)
    'initial_input': initial_input,
    }

In [31]:
for channel in channel_list:
    #
    print('')
    print('channel='+str(channel))
    print('')

    # target units
    feat_size = feat_num
     # the unit in the center of feature map
    feature_mask = np.zeros(feat_size)
    #t_index = int(feat_size[2]/2)
    #y_index = int(feat_size[3]/2) # the unit in the center of feature map
    #x_index = int(feat_size[4]/2) # the unit in the center of feature map
    
    #feature_mask[0,channel, t_index, y_index, x_index] = 1
    feature_mask[0, channel] = 1

    #
    preferred_vid = generate_preferred(net, target_layer_list, feature_mask=feature_mask , **opts)

    # save the results
    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.mat'
    sio.savemat(os.path.join(save_path,save_name),{'preferred_vid':preferred_vid})

    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.avi'
    save_video(normalise_vid(clip_extreme_pixel(preferred_vid,pct=0.04)), save_name, save_path, fr_rate = 10)
    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.gif'
    save_gif(normalise_vid(clip_extreme_pixel(preferred_vid, pct=0.04), save_name, save_path, fr_rate = 150)


channel=0

iter=1; mean(abs(feat))=0.00287055;
iter=2; mean(abs(feat))=0.00372351;
iter=3; mean(abs(feat))=0.000596068;
iter=4; mean(abs(feat))=0.00414294;
iter=5; mean(abs(feat))=0.00637835;
iter=6; mean(abs(feat))=0.00781173;
iter=7; mean(abs(feat))=0.0122691;
iter=8; mean(abs(feat))=0.015661;
iter=9; mean(abs(feat))=0.0172775;
iter=10; mean(abs(feat))=0.0183065;
iter=11; mean(abs(feat))=0.0259708;
iter=12; mean(abs(feat))=0.0111986;
iter=13; mean(abs(feat))=0.0224884;
iter=14; mean(abs(feat))=0.0239889;
iter=15; mean(abs(feat))=0.021741;
iter=16; mean(abs(feat))=0.0215787;
iter=17; mean(abs(feat))=0.02533;
iter=18; mean(abs(feat))=0.0347291;
iter=19; mean(abs(feat))=0.0216513;
iter=20; mean(abs(feat))=0.0215882;
iter=21; mean(abs(feat))=0.0269685;
iter=22; mean(abs(feat))=0.029074;
iter=23; mean(abs(feat))=0.0295967;
iter=24; mean(abs(feat))=0.0293212;
iter=25; mean(abs(feat))=0.0220904;
iter=26; mean(abs(feat))=0.0234983;
iter=27; mean(abs(feat))=0.0430696;
iter=28; mean(abs(feat)

NameError: name 'save_gif' is not defined

In [30]:
exec_str_list

['fc']

In [14]:
model = net.module

In [15]:
model

ResNet(
  (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (iden): MyIdentity()
      (downsample): Sequential(
     

In [29]:
exec_str_list

['net.fc.register_forward_hook(hook)']